In [ ]:
import warnings
import pandas as pd
# , Range1d#, LabelSet
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, load_data, get_cycle_data, get_cycle_data2, figures)
import dmyplant2
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import arrow
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
from IPython.display import HTML, display
from pprint import pprint as pp
from datetime import datetime
import numpy as np
from bokeh.plotting import figure, output_file, show
import bokeh
%load_ext autoreload
%autoreload 2
# default warn => SettingWithCopyWarning
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=UserWarning)


#import dmyplant2

cred()
mp = MyPlant(3600)
Engine._list_cached_validations()
# mp._fetch_installed_base(); # refresh local installed fleet database


In [ ]:
def sfun(x):
    return all([("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned')])


fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(
    by="Engine ID", ascending=True).reset_index(drop='index')
fleet.T


In [ ]:
motor = fleet.iloc[3]
modes = ['undefined', 'OFF', 'MAN', 'AUTO']
success = [True, False]
#success = [True]
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [ ]:
e = Engine.from_fleet(mp, motor)
fsm = FSMOperator(e, p_from=e['Commissioning Date'], p_to=datetime.now())

In [ ]:
fsm.restore()

In [ ]:
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False)

In [ ]:
fsm.run2(silent=False)

In [ ]:
#fsm.store()

In [ ]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))
          ].reset_index(drop='index')
display(HTML(pd.DataFrame.from_dict(
    e.dash, orient='index').T.to_html(escape=False, index=False)))
# rda[startstopFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")


In [ ]:
rda[startstopFSM.run2filter_content].round(2).fillna('')


In [ ]:
rdd = rda[rda['loadramp'] < 0.0]
rdd[startstopFSM.run2filter_content].round(2).fillna('')


In [ ]:
fsm.results['starts'][0]


In [ ]:
#rdb = rda[(rda['mode'].isin(['AUTO','MAN'])) & ((rda['count_alarms'] > 0) | (rda['count_warnings'] > 0))]
#rdb = rda[rda['synchronize'] < 15.0]
rdb = rda
#rdb[startstopFSM.run2filter_content].sort_values(by=["synchronize","loadramp"], ascending=[True,True]).round(2)


In [ ]:
vec = ['startpreparation', 'speedup', 'idle', 'synchronize', 'loadramp', 'targetload',
       'ramprate', 'cumstarttime', 'targetoperation', 'rampdown', 'coolrun', 'runout']
display(_=rdb[vec].hist(bins=30, figsize=(20, 20)))
# ax_list[0][2].set_xlim((0,10))
# display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))


In [ ]:
rde = rda[(rda.starttime > fsm._e['Commissioning Date'])
          & (rda['success'])].copy()
rde['datetime'] = pd.to_datetime(rde['starttime'])
dfigsize = (18, 12)
dset = [
    {'col': ['cumstarttime'], 'ylim':(-400, 900), 'color':'darkblue'},
    {'col': ['synchronize'], 'ylim':(-20, 300)},
    {'col': ['startpreparation'], 'ylim':(-600, 300)},
    {'col': ['hochlauf'], 'ylim':(-100, 200), 'color':'orange'},
    {'col': ['loadramp'], 'ylim':(-150, 900), 'color':'red'},
    {'col': ['ramprate'], 'ylim':(-3, 3)},
    {'col': ['no'], 'ylim':(-100, 10000), 'color':'black'},
    {'col': ['targetload'], 'ylim':(500, 5000)},
    {'col': ['runout'], 'ylim':(0, 100), 'color':'orange'},
]
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize, title=ftitle)
bokeh_show(fig)
del(fig)


In [ ]:
vset = ['Various_Values_SpeedAct', 'Power_PowerAct',
        'Hyd_PressCrankCase', 'Hyd_PressOilDif', 'Hyd_PressOil', 'Hyd_TempOil']
dset = [
    {'col': ['Power_PowerAct'], 'ylim':(0, 5000), 'color':'red'},
    {'col': ['Various_Values_SpeedAct'], 'ylim': [0, 2500], 'color':'blue'},
    {'col': ['Hyd_PressCrankCase'], 'ylim': [-100, 100], 'color':'orange'},
    {'col': ['Hyd_PressOilDif'], 'ylim': [0, 2], 'color':'black'},
    {'col': ['Hyd_PressOil'], 'ylim': [0, 10], 'color':'brown'},
    {'col': ['Hyd_TempOil'], 'ylim': [0, 110], 'color':'#2171b5'}
]


In [ ]:
rdb[startstopFSM.run2filter_content].sort_values(
    by=["synchronize", "loadramp"], ascending=[True, True]).round(2).fillna('')


In [ ]:
def f(x=0):
    global vv
    vv = x
    startversuch = rdb.iloc[x]
    ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
    display(HTML(ftitle))
    disp_result(startversuch)


interact(f, x=IntSlider(
    0, 0, rdb.shape[0]-1, 1, layout=widgets.Layout(width='90%')));


In [ ]:
# PLotter
lfigures = figures(fsm._e)
for f in lfigures:
    print(f, end=', ')
data = pd.DataFrame([])
#plotdef = ['actors','lubrication']
plotdef = list(lfigures.keys())
vset = []
for p in plotdef:
    dset = lfigures[p]
    vset += cvset(mp, dset)
vset = list(set(vset))
startversuch = rdb.iloc[vv]
dmaxlength = None
dminlength = None
#dmaxlength = 1200
#dminlength = 200000
data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength,
                       min_length=dminlength, cycletime=1, silent=True, p_data=vset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))

# pp(vset)
for doplot in plotdef:
    # for doplot in ['actors']:
    dset = lfigures[doplot]
    try:
        fig = FSM_splot(fsm, startversuch, data, dset, figsize=(18, 10))
        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)


In [ ]:
sv = startversuch.to_dict()
x0 = sv['startstoptiming']['loadramp'][-1]['start']
y0 = 0.0
default_ramp_duration = 100.0 / fsm._e['rP_Ramp_Set']
x1 = x0 + pd.Timedelta(default_ramp_duration, unit='sec')
y1 = fsm._e['Power_PowerNominal']
x0, x1, y0, y1, default_ramp_duration


In [ ]:
from bokeh.models import Span
ramp = fig.line(x=[x0, x1], y=[y0, y1], y_range_name='0',
                line_color='green', line_dash='dotted', line_alpha=0.4, line_width=4)
fig.add_layout(ramp)

c = fig.circle(x0, y0)
fig.add_layout(c)

this = Span(location=x0, dimension='height', y_range_name='0',
            line_color='blue', line_dash='solid', line_alpha=0.4, line_width=4)
fig.add_layout(this)

bokeh_show(fig)


In [ ]:
fsm.starts.iloc[vv].to_dict();

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)
